In [2]:
import requests
import time
import json
from datetime import datetime, timezone, timedelta
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.static import players

# Constants
JSON_FILE = "all_shots.json"

# Load player data
def get_player_name(player_id):
    try:
        player_data = players.get_players()
        for player in player_data:
            if str(player_id) == str(player["id"]):  # Match player ID
                return player["full_name"]
        return "Unknown Player"
    except Exception as e:
        print(f"Error fetching player name: {e}")
        return "Unknown Player"

# Helper function to get the currently active game ID
def get_active_game_id():
    board = scoreboard.ScoreBoard()
    games = board.games.get_dict()
    now = datetime.now(timezone.utc)  # Current time in UTC
    
    for game in games:
        game_start_time = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc)
        game_end_time = game_start_time + timedelta(hours=3)  # Approximate game duration (3 hours)
        
        # Ensure the game is within the time window and has not finished
        if game_start_time <= now <= game_end_time and game['gameStatus'] == 2:  # Status 2 means "in progress"
            game_info = f"{game['awayTeam']['teamName']} vs. {game['homeTeam']['teamName']} @ {parser.parse(game['gameTimeUTC']).astimezone(tz=None)}"
            print(f"Active game found: {game_info}")
            return game['gameId']

    return None

# Fetch the current active game ID
GAME_ID = get_active_game_id()
if not GAME_ID:
    raise ValueError("No active NBA game is in progress right now.")

API_URL = f'https://cdn.nba.com/static/json/liveData/playbyplay/playbyplay_{GAME_ID}.json'

# Initialize local storage for shot attempts
try:
    with open(JSON_FILE, "r") as file:
        all_shots = json.load(file)
except FileNotFoundError:
    all_shots = []

# Function to fetch and filter all shot attempts
def fetch_all_shots():
    try:
        data = requests.get(API_URL).json()
        new_shots = [
            {
                "playerId": action.get("personId"),
                "playerName": get_player_name(action.get("personId")),
                "clock": action["clock"],
                "x": action["x"],
                "y": action["y"],
                "shotDistance": action.get("shotDistance"),
                "area": action.get("area"),
                "shotResult": action.get("shotResult"),
                "pointsTotal": action.get("pointsTotal"),
                "teamId": action.get("teamId"),
            }
            for action in data["game"]["actions"]
            if action.get("isFieldGoal") == 1  # Field goal attempt
        ]
        return new_shots
    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

# Event listener loop
def listen_for_shots():
    global all_shots
    print(f"Listening for all shot attempts in game ID {GAME_ID}...")
    while True:
        new_shots = fetch_all_shots()
        for shot in new_shots:
            if shot not in all_shots:  # Avoid duplicates
                all_shots.append(shot)
                print(f"New shot detected: {shot}")
        # Save updated shots to the JSON file
        with open(JSON_FILE, "w") as file:
            json.dump(all_shots, file, indent=4)
        time.sleep(2)  # Poll every 2 seconds for faster updates

# Start the event listener
listen_for_shots()


ValueError: No active NBA game is in progress right now.